In [139]:
from matrix_factorization import read

In [140]:
train,val,test,mapping = read()

In [141]:
import pystan
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set()  # Nice plot aesthetic
np.random.seed(101)

model = """
data {
    int<lower=1> N;
    int<lower=1> U;
    int<lower=1> D;
    int<lower=0,upper=U> uu[N];
    int<lower=0,upper=D> dd[N];
    int<lower=1> K;
    int<lower=0> y[N];
    real shape;
    real rate;
    int<lower = 0, upper = 1> run_estimation; 
}
parameters {
    vector[U] xi;
    vector[D] eta;
    matrix[U,K] theta;
    matrix[D,K] beta;
    matrix[N,K] z;
}
model {
    if(run_estimation==1){
        for (u in 1:U){
            xi[u] ~ gamma(shape, rate);     

            for (k in 1:K){
                theta[u,k] ~ gamma(shape, xi[u]);
            }

        }
        for (d in 1:D){
            eta[d] ~ gamma(shape, rate);
            for (k in 1:K){
                beta[d,k] ~ gamma(shape, eta[d]);
            }
        }

        for (n in 1:N){
            real poisson_mean = 0;
            for (k in 1:K){
                 poisson_mean += theta[uu[n],k] * beta[dd[n],k];
            }
            if (poisson_mean > 10){
                poisson_mean = 10;
            }
            target += poisson_lpmf(y[n]|poisson_mean);
        }
    }
}
generated quantities {
    vector[N] y_sim;
    vector[U] xi_sim;
    vector[D] eta_sim;
    matrix[U,K] theta_sim;
    matrix[D,K] beta_sim;
    for (u in 1:U){
        xi_sim[u] = gamma_rng(shape, rate);     
        for (k in 1:K){
            theta_sim[u,k] = gamma_rng(shape, xi_sim[u]);
        }
    
    }
    for (d in 1:D){
        eta_sim[d] = gamma_rng(shape, rate);
        for (k in 1:K){
            beta_sim[d,k] = gamma_rng(shape, eta_sim[d]);
        }
    }
    for (n in 1:N){
        real poisson_mean = 0;
        for (k in 1:K){
             poisson_mean += theta[uu[n],k] * beta[dd[n],k];
        }
        if (poisson_mean > 10){
            poisson_mean = 10;
        }
        y_sim[n] = poisson_rng(poisson_mean);
    }
}
"""

In [132]:
model = """
data {
    int<lower=1> N;
    int<lower=1> U;
    int<lower=1> D;
    int<lower=0,upper=U> uu[N];
    int<lower=0,upper=D> dd[N];
    int<lower=1> K;
    real shape;
    real rate;
}
generated quantities {
    vector[N] y_sim;
    vector[U] xi_sim;
    vector[D] eta_sim;
    matrix[U,K] theta_sim;
    matrix[D,K] beta_sim;
    for (u in 1:U){
        xi_sim[u] = gamma_rng(shape, rate);
        for (k in 1:K){
            theta_sim[u,k] = gamma_rng(shape, xi_sim[u]);
        }
    
    }
    for (d in 1:D){
        eta_sim[d] = gamma_rng(shape, rate);
        for (k in 1:K){
            beta_sim[d,k] = gamma_rng(shape, eta_sim[d]);
        }
    }
    for (n in 1:N){
        real poisson_mean = 0;
        for (k in 1:K){
             poisson_mean += theta_sim[uu[n],k] * beta_sim[dd[n],k];
        }
        if (poisson_mean > 10){
            poisson_mean = 10;
        }
        y_sim[n] = poisson_rng(poisson_mean);
    }
}
"""

In [142]:
# Compile the model
sm = pystan.StanModel(model_code=model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_24cd6d6f74988665a43bab6e7a112ea9 NOW.


In [143]:
U,D = val.shape
uu, dd = val.nonzero()
uu = uu + 1
dd = dd + 1
uu = list(uu)
dd = list(dd)
y = val.toarray()
y = y[y>0].astype(int)
y = list(y)

In [144]:
# Put our data in a dictionary
#data = {'U': U, 'D': D, 'y': y, 'uu':uu, 'dd':dd, 'N':len(uu), 'K': 15, 'shape':0.5, 'rate':5.0, 'run_estimation':0}
data = {'U': U, 'D': D, 'y': y, 'uu':uu, 'dd':dd, 'N':len(uu), 'K': 15, 'shape':0.5, 'rate':5.0, 'run_estimation':0}

In [145]:
# Train the model and generate samples
fit = sm.sampling(data=data, iter=1, chains=1, warmup=5, thin=1, seed=101, n_jobs = 1, algorithm="Fixed_param")

RuntimeError: Exception: poisson_rng: Rate parameter is -4.10752, but must be > 0!  (in 'unknown file name' at line 77)


In [137]:
fit['y_sim'].shape

(1, 5017)

In [ ]:
summary_dict = fit.summary()
df = pd.DataFrame(summary_dict['summary'], 
                  columns=summary_dict['summary_colnames'], 
                  index=summary_dict['summary_rownames'])

In [ ]:
alpha_mean, beta_mean = df['mean']['alpha'], df['mean']['beta']

# Extracting traces
alpha = fit['alpha']
beta = fit['beta']
sigma = fit['sigma']
lp = fit['lp__']

In [ ]:
len(beta)

In [ ]:
df

In [ ]:
def plot_trace(param, param_name='parameter'):
  """Plot the trace and posterior of a parameter."""
  
  # Summary statistics
  mean = np.mean(param)
  median = np.median(param)
  cred_min, cred_max = np.percentile(param, 2.5), np.percentile(param, 97.5)
  
  # Plotting
  plt.subplot(2,1,1)
  plt.plot(param)
  plt.xlabel('samples')
  plt.ylabel(param_name)
  plt.axhline(mean, color='r', lw=2, linestyle='--')
  plt.axhline(median, color='c', lw=2, linestyle='--')
  plt.axhline(cred_min, linestyle=':', color='k', alpha=0.2)
  plt.axhline(cred_max, linestyle=':', color='k', alpha=0.2)
  plt.title('Trace and Posterior Distribution for {}'.format(param_name))

  plt.subplot(2,1,2)
  plt.hist(param, 30, density=True); sns.kdeplot(param, shade=True)
  plt.xlabel(param_name)
  plt.ylabel('density')
  plt.axvline(mean, color='r', lw=2, linestyle='--',label='mean')
  plt.axvline(median, color='c', lw=2, linestyle='--',label='median')
  plt.axvline(cred_min, linestyle=':', color='k', alpha=0.2, label='95% CI')
  plt.axvline(cred_max, linestyle=':', color='k', alpha=0.2)
  
  plt.gcf().tight_layout()
  plt.legend()

In [ ]:
plot_trace(alpha)

In [ ]:
model = """
data {
  int N;
  int P; // number of categories to be estimated
  int y[N]; // outcomes
  int<lower = 0, upper = 1> run_estimation; // a switch to evaluate the likelihood
  real<lower = 0> prior_sd; // standard deviation of the prior on theta
}
parameters {
  vector[P-1] theta_raw;
}
transformed parameters {
  vector[P] theta;
  theta[1] = 0.0;
  theta[2:P] = theta_raw;
}
model {
  // prior
  theta_raw ~ normal(0, prior_sd);
  
  // likelihood, which we only evaluate conditionally
  if(run_estimation==1){
    y ~ categorical(softmax(theta));
  }
}
generated quantities {
  vector[N] y_sim;
  for(i in 1:N) {
    y_sim[i] = categorical_rng(softmax(theta));
  }
}
"""

In [ ]:
sm = pystan.StanModel(model_code=model)

In [ ]:
data = dict(N = 1000, P = 5, y = np.random.choice([1,2],size = 1000),run_estimation = 0,prior_sd = 100)
sim_out = sm.sampling(data = data)

In [ ]:
library(dplyr)

fake_data_matrix  <- sim_out %>% 
  as.data.frame %>% 
  select(contains("y_sim"))

summary_tbl <- apply(fake_data_matrix[1:5,], 1, summary)